# CNN

Convolution Neural Network. A sort of ANN which is mainly used for computer vision field like Image Recognition etc.

Specialised for extracting local features of input data. 

Also can be used for NLP

## Filter

aka Kernel or Window

## Padding

adding fixed values to the edges of data. 

## Stride

the size which the filter moves.

if stride be bigger, the output size be smaller.